In [ ]:
import gmsh
import os
import import_ipynb
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import mu_0
from fem_pre_processing import read_mesh
from fem_processing import gaussian_quadrature, matrices_assembly
from fem_pos_processing import fem_analysis as fem, graph_results as graph

# Project 2: Coaxial Longitudinal Problem 

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_longitudinal.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

# `uniform_current_density()`

In [ ]:
def uniform_current_density(cell):
    """
    Calcula a densidade de corrente uniforme nas regiões condutoras
    de um cabo coaxial.

    Parâmetros:
    - I: Corrente total (A)
    - a: Raio do condutor interno (m)
    - b: Raio interno da blindagem (m)
    - c: Raio externo da blindagem (m)

    Retorna:
    - J_inner: Densidade de corrente no núcleo (A/m²)
    - J_outer: Densidade de corrente na blindagem (A/m²)
    """
    cell_name = cell['material']['name']
    radii = {'a': 1e-2, 'b': np.sqrt(3) * 1e-2, 'c': 2e-2}  
    Icc = 1

    # Área do condutor interno
    area_inner = np.pi * radii['a']**2
    
    # Área da blindagem
    area_outer = np.pi * (radii['c']**2 - radii['b']**2)
    
    # Densidades de corrente
    if cell_name == 'core':
        Jz = Icc / area_inner
    elif cell_name == 'dielectric':
        Jz = 0
    elif cell_name == 'sheath':
        Jz = -Icc / area_outer

    return Jz

# `apply_physics()`

In [ ]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    for key, cell in cell_data.items():  
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []
        
        # Calcular o valor de 'a' de acordo com a física do problema
        mur = cell['material']['relative_magnetic_permeability']
        a = 1/(mur * mu_0)
        
        # Get the global coordinates of the nodes
        ai = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Get the Gauss points and weights
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

        # Adicionar a nova chave 'material' ao dicionário da célula
        for xik in gauss_points: 
            xg_e, yg_e = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xik)
            cell['stiffness_term'].append(a * np.eye(2))
            cell['mass_term'].append(1)           
        
            # Adicionar a nova chave 'source' ao dicionário da célula
            cell['source'].append(uniform_current_density(cell))

        # Dictionary with boundary nodes
        cell_data[key]['abc'] = {'type': None, 'conn_idx': None}

    return mesh_data

# `create_entire_domain()`

In [1]:
def create_entire_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    mesh_data = {}
    radii = {'a': 1e-2, 'b': np.sqrt(3) * 1e-2, 'c': 2e-2}  
    type, order = FINITE_ELEMENT
    
    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("coax_long")
    factory = gmsh.model.occ

    # Criar os loops de curva
    in_boundary = factory.addCircle(0, 0, 0, radii['a'])
    die_boundary = factory.addCircle(0, 0, 0, radii['b'])
    out_boundary = factory.addCircle(0, 0, 0, radii['c'])

    # Criar loops de curva
    in_loop = factory.addCurveLoop([in_boundary])
    die_loop = factory.addCurveLoop([die_boundary])
    out_loop = factory.addCurveLoop([out_boundary])

    # Criar superfícies entre os loops
    inner_conductor = factory.addPlaneSurface([in_loop])  
    dielectric = factory.addPlaneSurface([die_loop, in_loop])  
    outer_conductor = factory.addPlaneSurface([out_loop, die_loop])

    # Sincronizar a geometria
    factory.synchronize()

    # Adicionar grupos físicos para Dim=1
    gmsh.model.addPhysicalGroup(
        1, [out_boundary], tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])

    # Adicionar grupos físicos para superfícies (Dim=2)	    
    for i, SurfaceTag in enumerate([inner_conductor, dielectric, outer_conductor]):
        gmsh.model.addPhysicalGroup(2, [SurfaceTag], tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Gerar malha
    gmsh.option.setNumber("Mesh.SaveAll", 1)
    gmsh.model.mesh.setSize(gmsh.model.getEntities(dim=0), h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/coax_long_domain_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)
    
    gmsh.finalize()

    return mesh_data

# `create_quarter_domain()`

In [ ]:
def create_quarter_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    mesh_data = {}
    radii = {'a': 1e-2, 'b': np.sqrt(3) * 1e-2, 'c': 2e-2}
    type, order = FINITE_ELEMENT

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("coaxial_quarter_longitudinal")
    factory = gmsh.model.occ

    # Criar pontos para os arcos no quadrante superior direito
    p0 = factory.addPoint(0, 0, 0)  # Centro do círculo
    p_in_start = factory.addPoint(radii['a'], 0, 0)  # Início do arco interno
    p_in_middle = factory.addPoint(0, radii['a'], 0)  # Fim do arco interno
    p_die_start = factory.addPoint(radii['b'], 0, 0)  # Início do arco dielétrico
    p_die_middle = factory.addPoint(0, radii['b'], 0)  # Fim do arco dielétrico
    p_out_start = factory.addPoint(radii['c'], 0, 0)  # Início do arco externo
    p_out_middle = factory.addPoint(0, radii['c'], 0)  # Fim do arco externo

    # Criar arcos no quadrante superior direito
    arc_in = factory.addCircleArc(p_in_start, p0, p_in_middle)
    arc_die = factory.addCircleArc(p_die_start, p0, p_die_middle)
    arc_out = factory.addCircleArc(p_out_start, p0, p_out_middle)

    # Criar segmentos retos para conectar os arcos
    seg_in_x = factory.addLine(p0, p_in_start)
    seg_die_x = factory.addLine(p_in_start, p_die_start)
    seg_out_x = factory.addLine(p_die_start, p_out_start)
    seg_in_y = factory.addLine(p0, p_in_middle)
    seg_die_y = factory.addLine(p_in_middle, p_die_middle)
    seg_out_y = factory.addLine(p_die_middle, p_out_middle)
    symmetry_x = [seg_in_x, seg_die_x, seg_out_x]
    symmetry_y = [seg_in_y, seg_die_y, seg_out_y]

    # Criar loops de curva para cada região
    in_loop = factory.addCurveLoop([seg_in_x, arc_in, seg_in_y])
    die_loop = factory.addCurveLoop([seg_die_x, arc_die, seg_die_y, -arc_in])
    out_loop = factory.addCurveLoop([seg_out_x, arc_out, seg_out_y, -arc_die])

    # Criar superfícies preenchidas
    inner_conductor = factory.addPlaneSurface([in_loop])
    dielectric = factory.addPlaneSurface([die_loop])
    outer_conductor = factory.addPlaneSurface([out_loop])

    # Sincronizar geometria
    factory.synchronize()

    # Adicionar grupos físicos para Dim=1 (curvas)
    for i, LineTags in enumerate([[arc_out], symmetry_x]):
        gmsh.model.addPhysicalGroup(1, LineTags, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para Dim=2 (superfícies)
    for i, SurfaceTag in enumerate([inner_conductor, dielectric, outer_conductor]):
        gmsh.model.addPhysicalGroup(2, [SurfaceTag], tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Gerar malha
    gmsh.option.setNumber("Mesh.SaveAll", 1)
    gmsh.model.mesh.setSize(gmsh.model.getEntities(dim=0), h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/coax_quarter_long_domain_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)
    
    gmsh.finalize()

    return mesh_data

# `set_analytical_solution()`

In [ ]:
def set_analytical_solution(x, y):
    return 0, np.zeros((2,1))

# `plot_b_field()`

In [ ]:
def plot_b_field(FINITE_ELEMENT, mesh_data, Azh, domain='Entire', flag_contourf=False):
    # Extração de informações sobre o elemento finito
    type, order = FINITE_ELEMENT
    
    # Estrutura de dados
    xg, yg, Bx, By, B_abs = [], [], [], [], []

    # Get the Gauss points and weights
    gauss_points, _ = gaussian_quadrature.general_triangle_rule(m=1, p=1)

    for cell in mesh_data['cell'].values():
        # Get the global coordinates of the nodes
        aie = [mesh_data['nodes'][idx]['xg'] for idx in cell['conn']]
        
        for xik in gauss_points:
            # Coordenadas globais no ponto de Gauss
            xge, yge = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, aie, xik)

            # Gradiente da solução aproximada u_h no ponto de Gauss
            grad_uh = fem.grad_uh_at_xik(FINITE_ELEMENT, mesh_data, cell, xik, Azh)

            # Rotacional do potencial vetor magnético
            Bfield = np.array([[0, -1], [1, 0]]) @ grad_uh

            # Acumular listas para visualização
            xg.append(xge)
            yg.append(yge)
            Bx.append(Bfield[0])
            By.append(Bfield[1])
            B_abs.append(np.linalg.norm(Bfield))

    # Visualização do campo magnético
    plt.figure(figsize=(10, 8))

    # Plotar o campo magnético
    plt.quiver(xg, yg, Bx, By, angles='xy', scale_units='xy',
                scale=0.03, color='blue', zorder=1)
    if flag_contourf:
        plt.tricontourf(xg, yg, B_abs, levels=50, cmap='Greys', alpha=0.6, zorder=0)
        plt.colorbar(label=r'$|\mathbf{B}| (T)$')
    
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    plt.axis('equal')
    
    # Salvando o arquivo no formato SVG
    filepath = graph.get_dir(f"pos_processing/pictures/b_field_{type}{order}_{domain}Domain.svg")
    print(f"Arquivo salvo em: {filepath}")
    plt.savefig(filepath, format="svg")
    plt.close()

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.